# Pandas Worked Example 09 / 17 / 2020

We start by importing the packages we will be using:
- Matplotlib - Creating simple line plots
- Seaborn - Generating statistical plots
- Numpy in case we need to set any dtypes
- Pandas for working with the data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


Now we read in our dataset (JHU COVID Timeseries data for Confirmed cases in the US). The url will always load will always be the most uptodate version. 

We now should do any data cleaning we want to do in the same cell.
- This will ensure that we don't run into any errors by reruning cells that set indexes etc.
- Drop any columns that have no data: `df.isnull().sum().sort_values()`
  - to drop if there are columns: `df = df.drop(columns=['col1', 'col2'])`
- Create a long version of the data : `df.melt(id_vars=index_cols, var_name='Date', value_name='Count')`
  - Set the dtype for Date: `df_l['Date'] = pd.to_datetime(df_l['Date'])`
- Set the index: `.set_index()`

 

In [ ]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

index_cols = ["UID", "iso2", "iso3", "code3", "FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Combined_Key"]

# Read the url into a dataframe
df = pd.read_csv(url)
# Convert the wide format data into a long format dataframe
df_l = df.melt(id_vars=index_cols, var_name='Date', value_name='Count')
# Set the date columns as a datetime to make looking at date ranges possible
df_l['Date'] = pd.to_datetime(df_l['Date'])
# Set all non - count columns to be in the index
df = df.set_index(index_cols)
# Output the first 5 rows of the dataframe
df.head()

# Working with the wide format data

Next we should look at what our DataFrame is telling us.

What columns do we have available?
What does each row represent?

Steps:
- Plot all the data
  - As a line plot
  - As a heatmap
    - Use these plots to find our next question

- Considerations:
  - Should we start at a fine granular level (individual cities) or at a higher level (States)
  - Should we transform the data at all?
    - Normalize, Standardise, Log Transform 

In [ ]:
# We need to use .index.get_level_values as 'Admin2' is now a index column
  # Find all cities called "Baldwin" in the DataFrame
f1 = df.index.get_level_values("Admin2") == "Baldwin"
# We could also filter by the state if we wanted to
# f2 = df.index.get_level_values("Province_State") == "Alabama"

# Apply our filters to the rows to subset the DataFrame
tmp = df.loc[f1]

# Set up the dimensions of the figure to make it larger
  # figsize=(width, height) (in inches)
  # dpi=int; The numner of dots per inch in the figure
fig = plt.figure(figsize=(8, 2), dpi=300)

# Add the first "Baldwin" data to the plot
_ = plt.plot(tmp.to_numpy().T[:,0])
# Add the second "Baldwin" cities data to the plot
_ = plt.plot(tmp.to_numpy().T[:,1])
# plt.show() will display the plot and then reset the figure for the next plot
plt.show()

In [ ]:
# Set our figure size
fig = plt.figure(figsize=(8, 2), dpi=300)

# Draw a line plot
_ = plt.plot(
    # Use all the data, the original data has a town per row, plot() wants a town per column so we transpose
    df.to_numpy().T,
    # Set the color of the lines
    c='black',
    # Set the thickness of the lines
    lw=0.2,
    # Set how seethrough the lines are. 0 transparent, 1 opaque
    alpha=0.2
    )


In [ ]:
# Finding the city with the highest confirmed cases
# Take the last value, and find the row that contains it
df.loc[df["9/16/20"] == df["9/16/20"].max()]

In [ ]:
# A heatmap requires a matrix of values as input
  # As we set all non value columns to be the index, we can just pass the dataframe to the heatmap method
sns.heatmap(df)

# Working with the long format data

One of the limiting factors when using the wide format data is that we cannot look at date ranges

- Filtering using the Date:
  - `date_filter = ('2020-03-01' <= df_l['Date']) & (df_l['Date'] <= '2020-03-31')`


In [ ]:
# Once a column is a datetime type you can use date strings to set ranges.
# Timedeltas is another option
#  You would use a timedelta to see the progression one month after the first case
#     - I would answer a question like this by:
#       1.  Find the date of first occurance for each town using groupby and min()
#       2.  Use that date in conjunction with a 1 month timedelta to set the range 

# I added .copy here to remove the error we got in the next cell
tmp = df_l.copy().loc[('2020-03-01' <= df_l['Date']) & (df_l['Date'] <='2020-03-31')]

tmp.head()

In [ ]:
# We use apply and numpys log function to log each count
#   The log of 0 is undefinded, so we add one to each count to avoid this
# The result is stored in a new column in tmp called 'logged'
tmp['logged'] = tmp['Count'].apply(lambda x: np.log(x + 1))
tmp.head()

In [ ]:
# WE can set the figure size for seaborn plots as well using this command
fig = plt.figure(figsize=(8, 2), dpi=300)
# This generates a boxplot using the State on the x axis and the logged Count on the y axis
_ = sns.boxplot(x='Province_State', y='logged', data=tmp)